In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime

In [80]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [81]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-07-07'
end_date = '2020-07-07'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [82]:
'''
    Функция навигации по данным из api response
    возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [83]:
'''
    Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)
    и вывод даты в строку

    delta = datetime.timedelta(days=1)
    day_current = datetime.date.fromisoformat(start_date) + delta
    date_str = datetime.date.strftime(day_current, '%Y-%m-%d')
'''

"\n    Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)\n    и вывод даты в строку\n\n    delta = datetime.timedelta(days=1)\n    day_current = datetime.date.fromisoformat(start_date) + delta\n    date_str = datetime.date.strftime(day_current, '%Y-%m-%d')\n"

In [95]:
# Делаем список данных за n дней

items_list = []

# n - количество запросов(дней)
n = 1

for i in range(1, n+1):
    current = get_url(url, 'next')
    url = current
    
    # Навигация по ссылкам, работаем с данными за один день
    url_level_0 = urllib.request.urlopen(url)
    data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
    
    # Cписок элементов за один день
    data_days = data_level_0.near_earth_objects[3]

    # Цикл по элементам списка за один день
    for data_day_item in data_days:
        dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
        dict_close_approach_data = data_day_item['close_approach_data']
        dict_relative_velocity = dict_close_approach_data[0]['relative_velocity']
        dict_miss_distance = dict_close_approach_data[0]['miss_distance']

        df1 = pd.DataFrame(data_day_item, index=['1'])
        df2 = pd.DataFrame(dict_estimated_diameter, index=['1'])
        df3 = pd.DataFrame(dict_close_approach_data, index=['1'])
        df4 = pd.DataFrame(dict_relative_velocity, index=['1'])
        df5 = pd.DataFrame(dict_miss_distance, index=['1'])

        df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['links',
                                                        'nasa_jpl_url',
                                                        'neo_reference_id',
                                                        'estimated_diameter',
                                                        'close_approach_data',
                                                        'relative_velocity',
                                                        'miss_distance',
                                                        'miles_per_hour',
                                                        'astronomical',
                                                        'lunar',
                                                        'miles'
                                                        ], axis=1
        ).rename(columns={'kilometers': 'miss_distance_kilometers',
                        'estimated_diameter_min': 'estimated_diameter_km_min',
                        'estimated_diameter_max': 'estimated_diameter_km_max'})

        items_list.append(df_item.to_dict())

In [85]:
# Навигация по ссылкам
# Работаем с данными за один день 2020-07-07
url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

,links,element_count,near_earth_objects
next,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,15,NaN
prev,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,15,NaN
self,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,15,NaN
2020-07-08,NaN,15,[{'links': {'self': 'http://api.nasa.gov/neo/r...


In [86]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2242450?api_key=8Mbg6rxSagXNYxrUbC8ynidJUcLn9h4CC7ucuT21'},
 'id': '2242450',
 'neo_reference_id': '2242450',
 'name': '242450 (2004 QY2)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2242450',
 'absolute_magnitude_h': 14.55,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 3.2700543928,
   'estimated_diameter_max': 7.3120639125},
  'meters': {'estimated_diameter_min': 3270.0543928193,
   'estimated_diameter_max': 7312.0639124658},
  'miles': {'estimated_diameter_min': 2.0319169681,
   'estimated_diameter_max': 4.5435044654},
  'feet': {'estimated_diameter_min': 10728.5252541373,
   'estimated_diameter_max': 23989.7117665742}},
 'is_potentially_hazardous_asteroid': True,
 'close_approach_data': [{'close_approach_date': '2020-07-08',
   'close_approach_date_full': '2020-Jul-08 11:27',
   'epoch_date_close_approach': 1594207620000,
   'relative_velocity': {'kilometers_per_second': '26.6793702848',
    'ki

In [87]:
# Создаем словари с выбранными данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data']
dict_relative_velocity = dict_close_approach_data[0]['relative_velocity']
dict_miss_distance = dict_close_approach_data[0]['miss_distance']

In [88]:
# Создаем DataFrame
# Объединяем DataFrame
df1 = pd.DataFrame(data_day_item, index=['1'])
df2 = pd.DataFrame(dict_estimated_diameter, index=['1'])
df3 = pd.DataFrame(dict_close_approach_data, index=['1'])
df4 = pd.DataFrame(dict_relative_velocity, index=['1'])
df5 = pd.DataFrame(dict_miss_distance, index=['1'])
df1.join(df2).join(df3).join(df4).join(df5)

,links,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,relative_velocity,miss_distance,orbiting_body,kilometers_per_second,kilometers_per_hour,miles_per_hour,astronomical,lunar,kilometers,miles
1,NaN,2242450,2242450,242450 (2004 QY2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2242450,14.55,NaN,True,"{'close_approach_date': '2020-07-08', 'close_a...",False,...,"{'kilometers_per_second': '26.6793702848', 'ki...","{'astronomical': '0.1631740078', 'lunar': '63....",Earth,26.6793702848,96045.7330254319,59679.0833901635,0.1631740078,63.4746890342,24410484.006243386,15167971.4132992868


In [94]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['links',
                                                  'nasa_jpl_url',
                                                  'neo_reference_id',
                                                  'estimated_diameter',
                                                  'close_approach_data',
                                                  'relative_velocity',
                                                  'miss_distance',
                                                  'miles_per_hour',
                                                  'astronomical',
                                                  'lunar',
                                                  'miles'
                                                  ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                  'estimated_diameter_min': 'estimated_diameter_km_min',
                  'estimated_diameter_max': 'estimated_diameter_km_max'})
df_item.to_dict()

{'id': {'1': '54340316'},
 'name': {'1': '(2023 BD7)'},
 'absolute_magnitude_h': {'1': 25.23},
 'is_potentially_hazardous_asteroid': {'1': False},
 'is_sentry_object': {'1': False},
 'estimated_diameter_km_min': {'1': 0.0239086457},
 'estimated_diameter_km_max': {'1': 0.0534613571},
 'close_approach_date': {'1': '2020-07-09'},
 'close_approach_date_full': {'1': '2020-Jul-09 16:38'},
 'epoch_date_close_approach': {'1': 1594312680000},
 'orbiting_body': {'1': 'Earth'},
 'kilometers_per_second': {'1': '12.5968407172'},
 'kilometers_per_hour': {'1': '45348.6265818552'},
 'miss_distance_kilometers': {'1': '21940006.350605802'}}